In [1]:
import pandas as pd
import yfinance as yf
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

In [2]:
# Load and preprocess data
def preprocess_data():
    data = yf.download("EURUSD=x", period="5d", interval="1h")
    data.index = pd.to_datetime(data.index)
    data = data.drop(columns=['Adj Close', 'Volume'])
    data
    
    # Initialize Bollinger Bands Indicator
    indicator_bb = BollingerBands(close=data["Close"], window=20, window_dev=2)
    data['bb_bbm'] = indicator_bb.bollinger_mavg()
    data['hband'] = indicator_bb.bollinger_hband()
    data['lband'] = indicator_bb.bollinger_lband()
    
    # Initialize RSI Indicator
    indicator_rsi = RSIIndicator(close=data["Close"], window=14)
    data['rsi'] = indicator_rsi.rsi()
    
    return data

In [3]:
# Define the trading strategy
def trading_strategy(data):
    position = 0  # 0 means no position, 1 means long, -1 means short
    entry_price = 0
    
    rsi_oversold = 30
    rsi_overbought = 70
    
    for i in range(1, len(data)):
        if position == 0:
            # Check for long entry
            if (data['rsi'].iloc[i] > rsi_oversold and data['rsi'].iloc[i-1] <= rsi_oversold) and (data['Close'].iloc[i] > data['bb_bbl'].iloc[i] and data['Close'].iloc[i-1] <= data['bb_bbl'].iloc[i-1]):
                position = 1
                entry_price = data['Close'].iloc[i]
                print(f"Entering long at {entry_price} on {data.index[i]}")
            # Check for short entry
            elif (data['rsi'].iloc[i] < rsi_overbought and data['rsi'].iloc[i-1] >= rsi_overbought) and (data['Close'].iloc[i] < data['bb_bbh'].iloc[i] and data['Close'].iloc[i-1] >= data['bb_bbh'].iloc[i-1]):
                position = -1
                entry_price = data['Close'].iloc[i]
                print(f"Entering short at {entry_price} on {data.index[i]}")
        elif position == 1:
            # Check for long exit
            if (data['rsi'].iloc[i] < rsi_overbought and data['rsi'].iloc[i-1] >= rsi_overbought) or (data['Close'].iloc[i] / entry_price - 1):
                position = 0
                cash += (data['Close'].iloc[i] - entry_price) * 10  # Assuming we buy/sell 10 units
                highest_value = max(highest_value, cash)
                lowest_value = min(lowest_value, cash)
                print(f"Exiting long at {data['Close'].iloc[i]} on {data.index[i]}")
        elif position == -1:
            # Check for short exit
            if (data['rsi'].iloc[i] > rsi_oversold and data['rsi'].iloc[i-1] <= rsi_oversold) or (entry_price / data['Close'].iloc[i] - 1):
                position = 0
                cash += (entry_price - data['Close'].iloc[i]) * 10  # Assuming we buy/sell 10 units
                highest_value = max(highest_value, cash)
                lowest_value = min(lowest_value, cash)
                print(f"Exiting short at {data['Close'].iloc[i]} on {data.index[i]}")
        
        data.at[data.index[i], 'cash'] = cash
        data.at[data.index[i], 'position'] = position
        data.at[data.index[i], 'portfolio_value'] = cash + position * (data['Close'].iloc[i] - entry_price) * 10
    
    return data

In [5]:

data = preprocess_data()
data  


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,bb_bbm,hband,lband,rsi
Datetime,,,,,,,,
2024-08-22 00:00:00+01:00,1.115947,1.116196,1.115698,1.115947,NaN,NaN,NaN,NaN
2024-08-22 01:00:00+01:00,1.115822,1.116071,1.114951,1.115076,NaN,NaN,NaN,NaN
2024-08-22 02:00:00+01:00,1.115324,1.115324,1.114703,1.114703,NaN,NaN,NaN,NaN
2024-08-22 03:00:00+01:00,1.114703,1.114827,1.114455,1.114703,NaN,NaN,NaN,NaN
2024-08-22 04:00:00+01:00,1.114703,1.114951,1.114703,1.114827,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-08-28 09:00:00+01:00,1.115947,1.116196,1.114206,1.114455,1.117233,1.120013,1.114452,34.858625
2024-08-28 10:00:00+01:00,1.114455,1.114827,1.113710,1.113958,1.117071,1.120197,1.113945,32.794643
2024-08-28 11:00:00+01:00,1.113958,1.113958,1.112471,1.112842,1.116866,1.120496,1.113236,28.685487
